Author: Kevin ALBERT  

Created: Oct 2020 

# FastAPI
_**samples on how to use FastAPI**_  

## Contents  
  * [install](#install)
  * [get_started](#get_started)
  * [path_arguments](#path_arguments)
  * [query_arguments](#query_arguments)
  * [basemodel](#basemodel)
  * [path_query_basemodel](#path_query_basemodel)
  * [response_model](#response_model)
  * [improving_models_and_documentation](#improving_models_and_documentation)
  * [todo_app](#todo_app)

|Method|Operation|Description|
|-|-|-|
|<b><span style="color:red">POST</span></b>|**C**reate|add a file<br>add a record|
|<b><span style="color:blue">GET</span></b>|**R**ead|retrieve information|
|<b><span style="color:orange">PUT</span></b>|**U**pdate|update existing resource|
|<b><span style="color:red">DELETE</span></b>|**D**elete|remove existing resource|

[REST API Tutorial](https://restfulapi.net/http-methods/) 

# install

In [1]:
# install python modules
! conda install -y --update-all -c conda-forge requests fastapi uvicorn

Solving environment: done

# All requested packages already installed.



In [2]:
from fastapi import FastAPI
import fastapi
import uvicorn
import requests
import json

In [3]:
# installed module versions
!conda -V
!python -V
print("requests", requests.__version__)
print("fastapi", fastapi.__version__)
print("uvicorn", uvicorn.__version__)
print("json", json.__version__)

conda 4.8.5
Python 3.8.5
requests 2.24.0
fastapi 0.61.1
uvicorn 0.11.3
json 2.0.9


### parameters

In [4]:
IPADDR           = "51.137.36.126" # this VM IP
PROJNAME         = "fastapi" # docker image name and project folder name
WEBAPP_PORT      = "5700"

### project folder

In [5]:
import os
project_folder = '../scripts/' + PROJNAME
os.makedirs(project_folder, exist_ok=True)
print(project_folder, 'folder created')

../scripts/fastapi folder created


# get_started

```python
app = FastAPI() # create an application instance 'app'

@app.get("/") # URL method 'GET' on URL path '/'
async def first_endpoint(): # better use async for speed
    return {"message": "Hello Data Science Learner"} # json data

if __name__ == '__main__':
    uvicorn.run("main:app",... # main.py:instance_name
```

In [6]:
%%writefile $project_folder/main.py
from fastapi import FastAPI
import uvicorn

app = FastAPI()

@app.get("/")
async def first_endpoint():
    return {"message": "Hello Data Science Learner"}

if __name__ == '__main__':
    uvicorn.run("main:app",  host="0.0.0.0", port=5700, reload=True)

Overwriting ../scripts/fastapi/main.py


**open shell, run script: (example)**
```sh
cd ~/notebooks/azuremachinelearning/code/scripts/fastapi/
conda activate py38_fastapi
python main.py

Uvicorn running ... (Press CTRL+C to quit)
```

In [7]:
print("openAPI swagger ==> http://"+IPADDR+":"+WEBAPP_PORT+"/docs")

openAPI swagger ==> http://51.137.36.126:5700/docs


In [8]:
# GET with path only
METHOD   = "GET" # {GET, POST, PUT, DELETE, PATCH}
URL      = "http://"+IPADDR+":"+WEBAPP_PORT

response = requests.request(METHOD, URL+"/") # HTTP request + service path
response.json()
# json.loads(response.text)
# json.dumps(response.json(), indent=2, ensure_ascii=False)

{'message': 'Hello Data Science Learner'}

# path_arguments

```python
@app.get("/people/{person_id}") # new URL path '/people/' + {argument} value
async def get_person(person_id): # function must use same argument name 'person_id' of type 'string'
    return {"person_id": person_id}

@app.get("/people/{person_id}")
async def get_person(person_id: int): # only accept a specific type {int, float, str, bool, ...}
    return {"person_id": person_id}
```

In [9]:
%%writefile $project_folder/main.py
from fastapi import FastAPI
import uvicorn

app = FastAPI()

@app.get("/")
async def first_endpoint():
    return {"message": "Hello Data Science Learner"}

@app.get("/people/{person_id}")
async def get_person(person_id: int):
    return {"message": f'person_id is {person_id}'}

if __name__ == '__main__':
    uvicorn.run("main:app",  host="0.0.0.0", port=5700, reload=True, debug=True)

Overwriting ../scripts/fastapi/main.py


In [10]:
# GET with path argument '0001':int
response = requests.request(METHOD, URL+"/people/0001")
response.json()

{'message': 'person_id is 1'}

In [11]:
# GET with path argument 'user01':str
response = requests.request(METHOD, URL+"/people/user01")
response.json()

{'detail': [{'loc': ['path', 'person_id'],
   'msg': 'value is not a valid integer',
   'type': 'type_error.integer'}]}

# query_arguments

```python
@app.get("/people")
async def list_people(surname: str, city: Optional[str]=None, limit: int=20): # query parameters defined in function not path
    return {"filters": {"surname": surname, "city": city}, "limit": limit}

firstname: str      # string, query, * required 
city: str=None,     # string, query,
limit: int=20):     # string, query, default 20
```

In [12]:
%%writefile $project_folder/main.py
from fastapi import FastAPI
import uvicorn

app = FastAPI()

@app.get("/")
async def first_endpoint():
    return {"message": "Hello Data Science Learner"}

@app.get("/people/{person_id}")
async def get_person(person_id: int):
    return {"message": f'person_id is {person_id}'}

@app.get("/people")
async def list_people(firstname: str, city: str=None, limit: int=20):
    return {"filters": {"firstname": firstname, "city": city}, "limit": limit}

if __name__ == '__main__':
    uvicorn.run("main:app",  host="0.0.0.0", port=5700, reload=True, debug=True)

Overwriting ../scripts/fastapi/main.py


In [13]:
# GET with "firstname" param only
response = requests.request(METHOD, URL+"/people?firstname=Kevin") # query, means using '?'
response.json()

{'filters': {'firstname': 'Kevin', 'city': None}, 'limit': 20}

In [14]:
# GET with "firstname" and "limit" params
response = requests.request(METHOD, URL+"/people?firstname=Kevin&limit=400") # AND, means using '&'
response.json()

{'filters': {'firstname': 'Kevin', 'city': None}, 'limit': 400}

In [15]:
# GET without params ("firstname" is *required)
response = requests.request(METHOD, URL+"/people")
response.json()

{'detail': [{'loc': ['query', 'firstname'],
   'msg': 'field required',
   'type': 'value_error.missing'}]}

# basemodel

```python
from typing import Optional
from pydantic import BaseModel

class Person(BaseModel):
    firstname: str         # string, required
    age: Optional[int]     # integer, not required
    city: str = "Brussels" # string, required, default "Brussels"

@app.post("/people")       # POST method
async def create_person(person: Person): # inherit BaseModel class type
    return person.dict()   # json data
```

In [16]:
%%writefile $project_folder/main.py
from typing import Optional
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

app = FastAPI()

class Person(BaseModel):
    firstname: str
    age: Optional[int]
    city: str = "Brussels"

@app.get("/")
async def first_endpoint():
    return {"message": "Hello Data Science Learner"}

@app.get("/people/{person_id}")
async def get_person(person_id: int):
    return {"message": f'person_id is {person_id}'}

@app.post("/people")
async def create_person(person: Person):
    return person.dict()

if __name__ == '__main__':
    uvicorn.run("main:app",  host="0.0.0.0", port=5700, reload=True, debug=True)

Overwriting ../scripts/fastapi/main.py


In [17]:
# POST age = "5"
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"firstname":"", "age":5, "city":""}

response = requests.request(METHOD, URL+"/people", json=DATA)
response.json()

{'firstname': '', 'age': 5, 'city': ''}

In [18]:
# POST age = "5x"
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"firstname":"", "age":'5x', "city":""}

response = requests.request(METHOD, URL+"/people", json=DATA)
response.json()

{'detail': [{'loc': ['body', 'age'],
   'msg': 'value is not a valid integer',
   'type': 'type_error.integer'}]}

In [19]:
# POST no age
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"firstname":"", "city":""}

response = requests.request(METHOD, URL+"/people", json=DATA)
response.json()

{'firstname': '', 'age': None, 'city': ''}

In [20]:
# POST no firstname
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"age":"", "city":""}

response = requests.request(METHOD, URL+"/people", json=DATA)
response.json()

{'detail': [{'loc': ['body', 'firstname'],
   'msg': 'field required',
   'type': 'value_error.missing'},
  {'loc': ['body', 'age'],
   'msg': 'value is not a valid integer',
   'type': 'type_error.integer'}]}

In [21]:
# POST no city
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"firstname":"", "age":5}

response = requests.request(METHOD, URL+"/people", json=DATA)
response.json()

{'firstname': '', 'age': 5, 'city': 'Brussels'}

In [22]:
# POST custom city name
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"firstname":"", "age":5, "city":"Antwerp"}

response = requests.request(METHOD, URL+"/people", json=DATA)
response.json()

{'firstname': '', 'age': 5, 'city': 'Antwerp'}

# path_query_basemodel

```python
@app.post("/people/{person_id}")   # either use as path
@app.post("/people")               # either use as query

async def create_person(person_id: int, person: Person): # combination of path/query and POST basemodel
    return {"person_id": person_id, **person.dict()}
```

In [23]:
%%writefile $project_folder/main.py
from typing import Optional
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

app = FastAPI()

class Person(BaseModel):
    firstname: str
    age: Optional[int]
    city: str = "Brussels"

@app.get("/")
async def first_endpoint():
    return {"message": "Hello Data Science Learner"}

@app.post("/people")
async def create_person(person_id: int, person: Person):
    return {"person_id": person_id, **person.dict()}

if __name__ == '__main__':
    uvicorn.run("main:app",  host="0.0.0.0", port=5700, reload=True, debug=True)

Overwriting ../scripts/fastapi/main.py


In [24]:
# POST with query method
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"firstname":"Kevin", "age":5, "city":"Antwerp"}
PARAMS = {'person_id': 1}
HEADERS = {'Content-Type': 'application/json'}

response = requests.request(METHOD, URL+"/people", json=DATA, params=PARAMS, headers=HEADERS)
response.json()

{'person_id': 1, 'firstname': 'Kevin', 'age': 5, 'city': 'Antwerp'}

# response_model

```python
# all data coming in:
class PersonIn(BaseModel):
    secret_key: str        # confidential data for example
    firstname: str         # same
    age: Optional[int]     # same
    city: str = "Brussels" # same

# all data coming out
class Person(BaseModel):
    firstname: str         # same
    age: Optional[int]     # same
    city: str = "Brussels" # same

# response_model         -> filter out confidential data
# response_model_include -> to include certain fields
# response_model_exclude -> to exclude certain fields
@app.post("/people", response_model=Person, response_model_include={"age", "firstname"})
async def create_person(person_id: int, person: PersonIn):
    return person # person_id no longer able to output
```

In [25]:
%%writefile $project_folder/main.py
from typing import Optional
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

app = FastAPI()

class PersonIn(BaseModel):
    secret_key: str
    firstname: str
    age: Optional[int]
    city: str = "Brussels"

class Person(BaseModel):
    firstname: str
    age: Optional[int]
    city: str = "Brussels"

@app.get("/")
async def first_endpoint():
    return {"message": "Hello Data Science Learner"}

@app.post("/people", response_model=Person, response_model_include={"age", "firstname"})
async def create_person(person: PersonIn):
    return person

if __name__ == '__main__':
    uvicorn.run("main:app",  host="0.0.0.0", port=5700, reload=True, debug=True)

Overwriting ../scripts/fastapi/main.py


In [26]:
# POST with query method
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"secret_key":"S3cr3tKey", "firstname":"Kevin", "age":5, "city":"Antwerp"}
PARAMS = {'person_id': 2}
HEADERS = {'Content-Type': 'application/json'}

response = requests.request(METHOD, URL+"/people", json=DATA, params=PARAMS, headers=HEADERS)
response.json()

{'firstname': 'Kevin', 'age': 5}

# improving_models_and_documentation

```python
# add a description as docstring
async def create_person(person: PersonIn):
    """A human being, with a name, age and city"""

from pydantic import BaseModel, Field
# add a Field object to define the field description, constraints and more:
    # name:str >=  1 (char_length) 
    # age:int  >= 18
    # city:str >=  1 
class Person(BaseModel):
    name: str = Field(...,
                      description="The firstname of this person",
                      example="Anna",
                      min_length=1)
    age: Optional[int] = Field(None,
                               description="The age of this person in years. Must be 18 years or older",
                               example=18,
                               ge=18)  # (g)reater or (e)qual 18
    city: str = Field("office",
                      description="The city this person work",
                      example="Brussels",
                      min_length=1)
```

In [27]:
%%writefile $project_folder/main.py
from typing import Optional
from fastapi import FastAPI
from pydantic import BaseModel, Field
import uvicorn

app = FastAPI()

class PersonIn(BaseModel):
    secret_key: str                    # *required
    firstname: str = Field(...,       # default: "" or ..., a value is *required
                           description="The firstname of this person",
                           example="Kevin",
                           min_length=1)
    age: Optional[int] = Field(None,   # default: None, due to Optional[int] a value is NOT required
                               description="The age of this person in years. Must be 18 years or older",
                               example=18,
                               ge=18)  # (g)reater or (e)qual 18
    city: str = Field("Brussel",       # default: "Brussel", if default: None then a value is *required !
                      description="The city this person works",
                      example="Antwerp",
                      min_length=1,
                      max_length=25)

class Person(BaseModel):
    firstname: str
    age: Optional[int]
    city: str

@app.get("/")
async def first_endpoint():
    return {"message": "Hello Data Science Learner"}

@app.post("/people", response_model=Person, response_model_include={ "firstname", "age", "city"})
async def create_person(person: PersonIn):
    """A human being, with a firstname, age and city"""
    return person

if __name__ == '__main__':
    uvicorn.run("main:app",  host="0.0.0.0", port=5700, reload=True, debug=True)

Overwriting ../scripts/fastapi/main.py


In [28]:
# POST no city, age above 18
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"secret_key":"S3cr3tKey", "firstname":"Steven", "age":25}
PARAMS = {}
HEADERS = {'Content-Type': 'application/json'}

response = requests.request(METHOD, URL+"/people", json=DATA, params=PARAMS, headers=HEADERS)
response.json()

{'firstname': 'Steven', 'age': 25, 'city': 'Brussel'}

In [29]:
# POST no city, no age, no name
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"secret_key":"S3cr3tKey"}
PARAMS = {}
HEADERS = {'Content-Type': 'application/json'}

response = requests.request(METHOD, URL+"/people", json=DATA, params=PARAMS, headers=HEADERS)
response.json()

{'detail': [{'loc': ['body', 'firstname'],
   'msg': 'field required',
   'type': 'value_error.missing'}]}

In [30]:
# POST age below 18, firstname < 1 char
METHOD = "POST"
URL = "http://"+IPADDR+":"+WEBAPP_PORT
DATA = {"secret_key":"S3cr3tKey", "firstname":"", "age":8}
PARAMS = {}
HEADERS = {'Content-Type': 'application/json'}

response = requests.request(METHOD, URL+"/people", json=DATA, params=PARAMS, headers=HEADERS)
response.json()

{'detail': [{'loc': ['body', 'firstname'],
   'msg': 'ensure this value has at least 1 characters',
   'type': 'value_error.any_str.min_length',
   'ctx': {'limit_value': 1}},
  {'loc': ['body', 'age'],
   'msg': 'ensure this value is greater than or equal to 18',
   'type': 'value_error.number.not_ge',
   'ctx': {'limit_value': 18}}]}

# todo_app

In [31]:
%%writefile $project_folder/main.py
from typing import Optional, List
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import uvicorn

app = FastAPI(title="Todo API")

class Todo(BaseModel):
    name: str
    due_date: str
    description: str

store_todo = []

@app.get('/')
async def index():
    return {"message": "Hello Data Science Learner"}

@app.post('/todo/')
async def create_todo(todo: Todo):
    store_todo.append(todo)
    return todo

@app.get('/todo/', response_model=List[Todo])
async def list_all_todos():
    return store_todo

@app.get('/todo/{id}')
async def get_todo(id: int):
    try:
        return store_todo[id]
    except:
        raise HTTPException(status_code=404, detail="Todo Not Found")

@app.put('/todo/{id}')
async def update_todo(id: int, todo: Todo):
    try:
        store_todo[id] = todo
        return store_todo[id]
    except:
        raise HTTPException(status_code=404, detail="Todo Not Found")

@app.delete('/todo/{id}')
async def delete_todo(id: int):
    try:
        obj = store_todo[id]
        store_todo.pop(id)
        return obj
    except:
        raise HTTPException(status_code=404, detail="Todo Not Found")

Overwriting ../scripts/fastapi/main.py
